## Assignment 10
#### CNN model
* Input (입력의 형태)
  + Input type: torch.Tensor
  + Input shape: (N x C x H x W)
    + N: Batch size, C: # of channels, H: height, W: width
  + (?, 1, 28, 28)
    + 여러장의, 흑백, 28x28 size의 이미지라고 가정하자
* Layer 설계
  + Layer 1
    + Conv2d >> C: 32, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    + ReLU
    + MaxPool >> Kernel size: 2, Stride: 2
입-출력 (?, 1, 28, 28) >> (?, 32, 14, 14)
  + Layer 2
    + Conv2d >> C: 64, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    + ReLU
    + MaxPool >> Kernel size: 2, Stride: 2
입-출력 (?, 32, 14, 14) >> (?, 64, 7, 7)
  + Layer 3
    + Linear >> input: 7x7x64 output: 10
    + Softmax

In [ ]:
# import
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 Input shape=(?, 1, 28, 28)
        #    Conv     -> (?, 32, 28, 28)
        #    Pool     -> (?, 32, 14, 14)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 Input shape=(?, 32, 14, 14)
        #    Conv      ->(?, 64, 14, 14)
        #    Pool      ->(?, 64, 7, 7)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 Input shape=(?, 64, 7, 7)
        #    Conv      ->(?, 128, 7, 7)
        #    Pool      ->(?, 128, 4, 4)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L4 Linear 128x4x4 inputs -> 10 outputs
        self.fc = torch.nn.Linear(128*4*4, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out